In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# California Alcohol Licenses
`abc.ca.gov-licenses-1.1.1` Last Update: 2020-11-26T19:23:48

__


This package is a conversion of the California Daily ABC file, so it is current as of the modification date of the package. The main change is the addition of the ``county`` variable, the FIPS code for the county, and the ``tract``, the ACS format geoid version of the tract identifier.


 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [ca_abc_licenses](notebooks/Process.ipynb#df)**. California Alcohol Beverage Control Licenses

## References
<ul><li> <strong><a href="https://www.abc.ca.gov/wp-content/uploads/WeeklyExport_CSV.zip#*.csv&start=2&headers=1">abc_licenses_source</a></strong>. Raw ABC data</li><ul>

In [3]:
df = pkg.reference('abc_licenses_source').dataframe()

In [4]:
df.head().T

,0,1,2,3,4
License Type,20,21,20,20,20
File Number,00257948,00446537,00515667,00203684,00165239
Lic or App,APP,APP,APP,APP,APP
Type Status,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE
Type Orig Iss Date,,,,,
Expir Date,,,,,
Fee Codes,P40,P40,P40,P20,P0
Dup Counts,,,,,
Master Ind,Y,Y,Y,Y,Y
Term in # of Months,12,12,12,12,12


In [5]:
from geoid.acs import County, Tract
from geoid.censusnames import geo_names

pkg = mp.open_package('http://s3.amazonaws.com/library.metatab.org/huduser.gov-zip_tract-2016-2.csv') 
zip_tract=pkg.resource('zip-tract').dataframe()

# Create a zip to tract map. Each zip can be in multiple tracts; this map will 
# use the tract that contains the highest portion of businesses. 
zt_map = {}
for idx, row in zip_tract.sort_values([ 'zip', 'bus_ratio']).iterrows():
    zt_map[row.zip] = row.tract


In [6]:
def find_county(n):
    for k,v in geo_names.items():
        if k[0] == 6 and n.lower() in v.lower():
            return n, k[1]
        
county_map = dict(find_county(c) for c in df['Prem County'].unique() if c and c.strip() )


df['county'] = df['Prem County'].replace(county_map)

def tract(r):
    try:
        a, b =  r['Prem Census Tract #'].split('.')
        return str(Tract(6,r.county, (a+b).zfill(6)))
    except:
        return None

df['tract'] = df.apply(tract, axis=1)


In [7]:
df

,License Type,File Number,Lic or App,Type Status,Type Orig Iss Date,Expir Date,Fee Codes,Dup Counts,Master Ind,Term in # of Months,...,DBA Name,Mail Addr 1,Mail Addr 2,Mail City,Mail State,Mail Zip,Prem County,Prem Census Tract #,county,tract
0,20,00257948,APP,ACTIVE,,,P40,,Y,12,...,CIRCLE K 457,15825 MOJAVE DR,,VICTORVILLE,CA,92394,RIVERSIDE,0451.20,65,14000US06065045120
1,21,00446537,APP,ACTIVE,,,P40,,Y,12,...,CIRCLE K 6004,15825 MOJAVE DR,,VICTORVILLE,CA,92394,MONTEREY,0106.04,53,14000US06053010604
2,20,00515667,APP,ACTIVE,,,P40,,Y,12,...,CIRCLE K 9476,15825 MOJAVE DR,,VICTORVILLE,CA,92394,SAN DIEGO,0170.43,73,14000US06073017043
3,20,00203684,APP,ACTIVE,,,P20,,Y,12,...,CIRCLE K 8587,15825 MOJAVE DR,,VICTORVILLE,CA,92394,LOS ANGELES,4003.04,37,14000US06037400304
4,20,00165239,APP,ACTIVE,,,P0,,Y,12,...,CIRCLE K 3608,15825 MOJAVE DR,,VICTORVILLE,CA,92394,FRESNO,0082.00,19,14000US06019008200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122076,21,00621003,APP,PEND,,,,,Y,12,...,,22 MICHAEL RD,,LADERA RANCH,CA,92694,,,,None
122077,47,00621006,APP,PEND,,,,,Y,12,...,,39626 10TH ST WEST,SUITE F,PALMDALE,CA,93551,,,,None
122078,47,00621005,APP,PEND,,,,,Y,12,...,,1626 N WILCOX,#185,LOS ANGELES,CA,90028-6206,,,,None
122079,47,00621011,APP,PEND,,,,,Y,12,...,,1626 N WILCOX,#185,LOS ANGELES,CA,90028-6206,,,,None
